In [7]:
from lxml.etree import HTML
from collections import defaultdict
import requests
import json
from domainer.config import DATA_DIR

In [8]:
# company processing logic
def process_element(el) -> dict:
    fields = defaultdict(list)
    main_div = el.xpath(".//div[@class='blog34_item']")[0]
    for item in main_div.xpath(".//*[@fs-cmsfilter-field]"):
        key = item.xpath("./@fs-cmsfilter-field")[0]
        value = item.xpath(".//text()")
        if value:
            fields[key] = fields[key] + value
    return {
        "name": main_div.xpath(".//h5/text()")[0].strip(),
        "website": main_div.xpath("./a/@href")[0],
    } | dict(fields)

In [9]:
companies = []

for i in range(1, 20):
    cookies = {
        "WG_CHOOSE_ORIGINAL": "1",
    }

    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "accept-language": "en-US,en;q=0.9",
        "cache-control": "no-cache",
        # 'cookie': 'WG_CHOOSE_ORIGINAL=1',
        "pragma": "no-cache",
        "priority": "u=0, i",
        "sec-ch-ua": '"Chromium";v="125", "Not.A/Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Linux"',
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "sec-gpc": "1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
    }

    params = {
        "a7511bde_page": str(i),
    }

    response = requests.get(
        "https://www.antler.co/portfolio",
        params=params,
        cookies=cookies,
        headers=headers,
    )
    tree = HTML(response.text)
    companies += [
        process_element(el)
        for el in tree.xpath(
            "//div[@role='listitem' and @class='w-dyn-item' and .//a[contains(@href, '/portfolio-companies')]]"
        )
    ]

len(companies)

1027

In [10]:
companies[0]

{'name': '12iD',
 'website': 'https://www.12iD.com',
 'location': ['Sweden'],
 'sector': ['Fintech'],
 'description': ['12iD provides a solution to issue digital identities and to identify users remotely and uniquely.']}

In [11]:
with open(DATA_DIR / "antler_final.json", "w") as f:
    json.dump(companies, f)